In [1]:
'''
This is just a little try on predicting wave numbers with material filed and eigenvalues
and is not yet trained and tested due to limited time.

Author:xyt
2019.8.8
'''
import tensorflow as tf 
import numpy as np
import keras
import keras.backend as K 
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, GlobalMaxPooling1D, MaxPooling2D,Input,Conv1D,MaxPooling1D,MaxPool2D
from keras.optimizers import Adam
#from keras.utils.training_utils import multi_gpu_model
import scipy as sp
import keras.backend as K
import h5py
import keras.backend.tensorflow_backend as KTF
 
KTF.set_session(tf.Session(config=tf.ConfigProto(device_count={'cpu':0})))

Using TensorFlow backend.


In [30]:
def multi_input_model():
    input1_= Input(shape=(128,128,3), name='material',dtype = "float32")
    input2_ = Input(shape=(50,1), name='eigenvalues',dtype = "float32")
 
    x1 = Conv2D(60, kernel_size=(2,2), strides=1, activation='relu', padding='same')(input1_)
    x1 = MaxPool2D(pool_size=(2,2), strides=32)(x1)
#     x1 = Conv2D(60, kernel_size=(2,2), strides=1, activation='relu', padding='same')(x1)
#     x1 = MaxPool2D(pool_size=(2,2), strides=4)(x1)
#     x1 = Conv2D(60, kernel_size=(2,2), strides=1, activation='relu', padding='same')(x1)
#     x1 = MaxPool2D(pool_size=(2,2), strides=2)(x1)    
    x1 = Flatten()(x1)
    
    x2 = Conv1D(16, kernel_size=3, strides=1, activation='relu', padding='same')(input2_)
    x2 = MaxPooling1D(pool_size=2, strides=2)(x2)
    
    x2 = Flatten()(x2)
    #x2 = K.transpose(x2)
    x = keras.layers.concatenate([x1, x2])
    #x = Flatten()(x)
    
    x = Dense(10, activation='relu')(x)
    output_ = Dense(2, activation='relu', name='output')(x)
 
    model = Model(inputs=[input1_, input2_], outputs=[output_])
    
    earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.1, patience=10, verbose=2, mode='auto')
    model.summary()
 
    return model



In [31]:
model = multi_input_model()
model.compile(loss="MSE", optimizer='adam', metrics=["MAE"])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
material (InputLayer)           (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
eigenvalues (InputLayer)        (None, 50, 1)        0                                            
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 128, 128, 60) 780         material[0][0]                   
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 50, 16)       64          eigenvalues[0][0]                
__________________________________________________________________________________________________
max_poolin

In [28]:
def training_data_generator(x_mean,x_range,y_mean,y_range):
    ds = h5py.File("../data/data2D_gzipped_famfiles_%d.h5","r",driver='family', memb_size=2500*10**6)
    for key1 in list(ds.keys()):
        x_count = 0
        y_count = 0
        for key2 in list(ds[key1].keys()):   #0-399 or 0-199
            if key2[0]!="x" or ds[key1][key2].shape[0]>64:
                continue

            x_raw = ds[key1][key2][:]
            y_key = "y"+key2[1:]
            y_raw = ds[key1][y_key][:]
            x_normed = (x_raw-x_mean)/x_range
            y_normed = (y_raw-y_mean)/y_range

            yield([x_normed[:,:,:,0:3],y_normed.reshape(y_normed.shape+(1,))],x_normed[:,0,0,3:5])
                

In [32]:
x_mean = [[1.49880835e+11, 4.40176392e+03 ,2.74842257e-01 ,2.49137578e-01,2.49167977e-01]]
x_range =[2.99999999e+11, 8.00000000e+03 ,4.49999999e-01 ,4.99999586e-01,4.99985816e-01]
y_mean = 1096210.1699824824
y_range = 6727841.0

model.fit_generator(training_data_generator(x_mean,x_range,y_mean,y_range),
                    steps_per_epoch=999,epochs=2)

Epoch 1/2
 56/999 [>.............................] - ETA: 51:52 - loss: 0.0835 - mean_absolute_error: 0.2505

KeyboardInterrupt: 